In [1]:
from requests import get
from datetime import datetime, timezone
import re
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import operator

# popular streamers
streamnames = { "xqc": ["xqc"], "summit": ["summit"], "tyler1": ["tyler", "t1"], "mizkif": ["miz", "mizkif"], "erobb": ["erobb", "wrobb", "lrobb"], "soda": ["soda", "chance morris"],
                "josh": ["josh"], "forsen": ["forsen"], "destiny": ["destiny"], "doc": ["doc", "drdisrespect"], "pokelawls": ["poke", "pokelawls"], "pokimane": ["poki", "pokimane"],
                "mitch": ["mitch"], "hassan": ["hassan", "hasan", "hasanabi"], "asmon": ["asmon"], "trainwreck": ["train"], "alinity": ["alinity"], "greek": ["greek"], "maya": ["maya"],
                "anita": ["anita"], "lirik": ["lirik"], "shroud": ["shroud"], "ice": ["^ice", " ice" "cx"], "moon": ["moon"], "qt": ["qt"], "rajj": ["rajj"], "ninja": ["ninja"]}

allDates = [datetime(2019,4,30), datetime(2019,3,31), datetime(2019,2,28), datetime(2019,1,31), datetime(2018,12,31), datetime(2018,11,30), datetime(2018,10,31),
                datetime(2018,9,30), datetime(2018,8,31), datetime(2018,7,31), datetime(2018,6,30), datetime(2018,5,31), datetime(2018,4,30), datetime(2018,3,31),
                datetime(2018,2,28), datetime(2018,1,31), datetime(2017,12,31), datetime(2017,11,30), datetime(2017,10,31), datetime(2017,9,30), datetime(2017,8,31),
                datetime(2017,7,31), datetime(2017,6,30), datetime(2017,5,31), datetime(2017,4,30), datetime(2017,3,31), datetime(2017,2,28), datetime(2017,1,31),
                datetime(2016,12,31)]
streamcount = {key: 0 for key in streamnames}
topOfMonth = []

In [3]:
date = datetime.utcnow()
timeUTC = time.mktime(date.timetuple())
toAdd = get('https://api.pushshift.io/reddit/search/submission/?subreddit=LivestreamFail&sort=desc&sort_type=created_utc&before=' + str(timeUTC)[:-2] + '&limit=500')
for aDate in allDates:
    timestamp = time.mktime(aDate.timetuple())
    nameDate = str(date)[:7]
    while datetime.fromtimestamp(timestamp) < date:
        title = ""
        score = 0
        timeUTC = 0

        post = toAdd.text
        splitpost = post.split("\"author\": ")

        for item in splitpost:
            if re.search("\"score\"", item) is not None:
                score = re.search("\"score\": .*", item).group(0)
                score = score[9:-1]
            if re.search("\"title\"", item) is not None:
                title = re.search("\"title\": .*\"", item).group(0)
                title = title[10:-1].lower()
            if re.search("\"created_utc\"", item) is not None:
                created_utc = re.search("\"created_utc\": .*", item).group(0)
                timeUTC = created_utc.split(" ")
                timeUTC = timeUTC[-1][:-1]
                
            date = datetime.utcfromtimestamp(float(timeUTC))

            for aname in streamnames:
                for name in streamnames[aname]:
                    if re.search(name, title) is not None:
                        streamcount[aname] = streamcount[aname] + 1

        if re.search("\\.", str(timeUTC)) is not None:
            timeUTC = str(timeUTC)[:-2]
        toAdd = get('https://api.pushshift.io/reddit/search/submission/?subreddit=LivestreamFail&sort=desc&sort_type=created_utc&before=' + str(timeUTC) + '&limit=500')
        #print(toAdd) 

    # building plot
    plt.rcdefaults()
    fig, ax = plt.subplots()
    peoplelist = []
    popularity = []
    for key in streamcount:
        peoplelist.append(key)
        popularity.append(streamcount[key])

    y_pos = np.arange(len(peoplelist))

    ax.barh(y_pos, popularity, align='center')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(peoplelist)
    ax.invert_yaxis()
    ax.set_xlabel('Mentions in Titles')
    ax.set_title('Streamer Popularity on /r/LiveStreamFail for ' + nameDate )

    # ensures bad dates are not used 
    if date.year < datetime(2015,1,1).year:
        continue

    plt.savefig(str(nameDate+'.png'))

    topOfMonth.append(max(streamcount.items(), key=operator.itemgetter(1))[0])
    streamcount = {key: 0 for key in streamcount}

    plt.close()

print(topOfMonth)

['mizkif', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'xqc', 'greek', 'ice', 'greek', 'greek', 'greek', 'greek', 'greek', 'mitch', 'soda', 'forsen', 'tyler1', 'doc', 'ice', 'greek', 'lirik', 'doc', 'tyler1', 'tyler1']
